In [2]:
import torch
import dqc
import dqc.xc
import dqc.utils

In [3]:
class MyLDAX(dqc.xc.CustomXC):
    def __init__(self, a, p):
        super().__init__()
        self.a = a
        self.p = p

    @property
    def family(self):
        # 1 for LDA, 2 for GGA, 4 for MGGA
        return 1

    def get_edensityxc(self, densinfo):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc(densinfo.u * 2) + self.get_edensityxc(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return self.a * rho ** self.p
        
    def get_edensityxc_derivative(self, densinfo, number_of_parameter):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc_derivative(densinfo.u * 2, number_of_parameter) 
                          + self.get_edensityxc_derivative(densinfo.d * 2, number_of_parameter))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            if number_of_parameter == 0: # parameter a
                return rho ** self.p
            elif number_of_parameter == 1: # parameter p
                return self.a * rho ** (self.p - 1)

In [4]:
a = torch.nn.Parameter(torch.tensor(1.0, dtype=torch.double))
p = torch.nn.Parameter(torch.tensor(2.0, dtype=torch.double))
myxc = MyLDAX(a, p)

In [5]:
mol = dqc.Mol(moldesc="N -1 0 0; N 1 0 0", basis="3-21G")
qc = dqc.KS(mol, xc=myxc).run()
ene = qc.energy()
print(ene)

tensor(-54.0932, dtype=torch.float64, grad_fn=<AddBackward0>)


/home/radium/miniconda3/lib/python3.9/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=2.508e-02, |f|=2.288e-01 at iter 32
  warnings.warn(ConvergenceWarning(msg))


In [6]:
dm = qc._dm.detach().clone() # density matrix
dm

tensor([[ 4.4449e-02,  2.4137e-14,  1.0511e-01, -5.0424e-02,  1.0289e-01,
          2.2958e-14, -5.8054e-02,  8.7262e-15, -1.4612e-14, -4.7588e-14,
          1.2283e-01,  8.2387e-02,  1.9097e-01, -6.7452e-14,  3.2144e-02,
          1.8733e-14,  1.1592e-15,  1.0015e-13],
        [ 2.4137e-14,  9.9451e-02,  1.0472e-13, -6.4384e-14,  5.0526e-14,
         -1.8880e-02, -6.3688e-15,  1.1943e-06,  2.0303e-04, -1.8050e-01,
          1.5046e-13,  6.6087e-14,  2.0816e-13,  6.9983e-02, -9.8086e-14,
         -4.7394e-03,  1.2609e-03, -3.8878e-01],
        [ 1.0511e-01,  1.0472e-13,  4.0383e-01,  3.7858e-02,  1.9606e-01,
          3.0889e-14, -8.9278e-02,  5.3771e-14, -3.2877e-14, -9.8062e-14,
          1.4279e-01,  5.8679e-01,  4.2135e-01, -6.3006e-14, -2.0842e-01,
          6.0567e-14,  4.0853e-14,  8.9760e-14],
        [-5.0424e-02, -6.4384e-14,  3.7858e-02,  4.6050e-01, -9.5657e-02,
         -1.5449e-14,  4.3600e-02,  4.6571e-14,  4.2604e-14,  5.5748e-14,
         -6.9951e-02,  5.8823e-01, -5.7

In [13]:
fock = qc._engine.dm2scp(dm).clone()

In [10]:
nao = qc.get_system().get_hamiltonian().nao
norb = qc._engine.norb

coeff = qc.get_system().get_hamiltonian().dm2ao_orb_params(dm, norb) # full C matrix
coeff

tensor([[-1.0950e-01,  8.7311e-04,  7.6791e-02, -1.0924e-03, -5.1973e-02,
         -2.7058e-02,  3.0051e-02],
        [ 1.7530e-02,  1.3151e-03, -2.2185e-03, -8.9175e-04,  3.0709e-02,
          8.7984e-02,  2.0181e-01],
        [-1.5374e-01,  1.3291e-01,  1.8973e-01, -2.5090e-02, -1.7868e-01,
         -2.6067e-01,  1.5530e-01],
        [ 1.7531e-01, -1.1001e-01, -2.3571e-01,  8.6145e-03,  1.6242e-02,
         -3.3918e-01,  1.2834e-01],
        [-3.0104e-01, -1.0579e-01,  1.3109e-01,  1.2810e-02, -9.9477e-02,
         -5.0955e-02,  6.5690e-02],
        [-9.2691e-02,  9.0151e-02, -1.0075e-01, -4.6311e-03,  3.6630e-02,
         -1.9666e-02, -3.7441e-02],
        [ 1.9154e-01,  1.0924e-01, -5.2329e-02, -1.4263e-02,  4.6669e-02,
          2.4073e-02, -3.5585e-02],
        [ 5.1077e-04,  6.0678e-03,  7.1713e-03,  5.4505e-02,  1.1161e-02,
         -4.5678e-03,  5.3177e-04],
        [-3.1171e-03, -2.5422e-03,  2.1986e-02, -1.3248e-02,  4.6678e-02,
         -1.4151e-02,  4.0380e-05],
        [-

$$ \frac{\partial E[\rho](\vec{\theta})}{\partial C_{bj}} = 2f_b \sum_i C_{bi}F_{ij}$$
so
$$ \frac{\partial E[\rho](\vec{\theta})}{\partial \textbf{C}} = 2\textbf{f}\textbf{C}\textbf{F}$$
$\textbf{f}$ here is matrix: $f_{ab}=\delta_{ab}f_a$

In [8]:
occ = torch.diag(qc._engine.orb_weight.detach().clone())
print(occ)

tensor([[2., 0., 0., 0., 0., 0., 0.],
        [0., 2., 0., 0., 0., 0., 0.],
        [0., 0., 2., 0., 0., 0., 0.],
        [0., 0., 0., 2., 0., 0., 0.],
        [0., 0., 0., 0., 2., 0., 0.],
        [0., 0., 0., 0., 0., 2., 0.],
        [0., 0., 0., 0., 0., 0., 2.]], dtype=torch.float64)


In [14]:
derivative = torch.matmul(torch.matmul(occ, torch.t(coeff)), fock)
derivative

tensor([[ 2.0347e-01, -2.0898e-01, -1.6000e-01,  1.8448e-01,  8.1549e-01,
          5.1087e-02, -6.5064e-01, -4.9838e-03,  1.2782e-02,  3.9851e-01,
          1.8976e+00,  8.8441e-03, -4.5361e-01, -1.1345e-01,  2.9121e+00,
          5.4752e-02,  4.4916e-03,  8.5159e-01],
        [ 2.9737e-01,  9.3438e-02,  1.7108e-01, -1.3157e-01,  1.1985e+00,
         -1.7721e-02, -8.6242e-01,  5.1583e-03, -1.4098e-02, -1.8663e-01,
          2.3568e+00,  6.6891e-02,  3.7276e-01, -3.1202e-02,  2.9067e+00,
         -2.7567e-02,  1.6170e-02, -4.5348e-01],
        [ 7.8551e-02, -8.2438e-02,  2.2120e-01, -2.6022e-01,  3.2087e-01,
          1.8078e-02, -1.8646e-01, -9.3194e-03,  1.3318e-02,  1.8078e-01,
          4.2647e-01, -2.4136e-02,  6.2075e-01,  2.9285e-02,  1.2785e-01,
         -2.7840e-02,  6.7243e-02,  4.8494e-01],
        [-4.1657e-02, -3.9082e-02, -3.1041e-02,  1.1239e-02, -1.6762e-01,
         -3.4573e-03,  1.2007e-01, -5.9984e-03, -3.0473e-03,  3.7344e-02,
         -3.2704e-01, -2.8455e-02, -5.0